In [110]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pylatexenc
from pylatexenc.latex2text import LatexNodes2Text
import string

In [111]:
# dat=pd.read_excel("./AuthorList.xlsx",engine='openpyxl')
dat=pd.read_excel("./KTS.xlsx",engine='openpyxl')
dat.index=dat.index+2

In [130]:
list_order = [1, 2, 3, 4]
# Convert 'list_order' column to categorical with specified list order
# dat['ListOrder'] = pd.Categorical(dat['ListOrder'], categories=list_order, ordered=True)

# Sort DataFrame by 'list_order' first, then alphabetically by 'alphabetical' column
dat = dat.sort_values(by=['ListOrder', 'LastName'])

dat

,LastName,Initial,Footnote,ListOrder,Institution1,Address1,Institution2,Address2
24,Heffner,M.,NaN,1,Lawrence Livermore National Laboratory,"7000 East Ave, Livermore CA 94550",NaN,NaN
4,Lenardo,B. G.,NaN,2,SLAC National Accelerator Laboratory,"7000 East Ave, Livermore CA 94550",NaN,NaN
2,Mistry,K.,NaN,3,The University of Texas at Arlington,"701 S Nedderman Dr, Arlington, TX 76019",NaN,NaN
3,Rogers,L.,NaN,4,Argonne National Lab,2575 Sand Hill Rd,NaN,NaN
31,Anker,A.,NaN,NaN,SLAC National Accelerator Laboratory,"2575 Sand Hill Rd, Menlo Park, CA 94025",NaN,NaN
6,Avasthi,A.,NaN,NaN,Case Western Reserve University,"10900 Euclid Ave, Cleveland, OH 44106",NaN,NaN
15,Brunner,T.,NaN,NaN,McGill University,"Department of Physics, 3600 University Street,...",NaN,NaN
25,Catarineu,N. R.,NaN,NaN,Lawrence Livermore National Laboratory,"7000 East Ave. Livermore, CA 94550",NaN,NaN
27,Cottle,A.,NaN,NaN,University College London,"Department of Physics & Astronomy, Gower St, L...",NaN,NaN
30,Englezos,P.,NaN,NaN,Rutgers University,"Piscataway, NJ, 08854, USA",NaN,NaN


In [113]:
# To add a corresponding author, use this format, indexed by excel sheet rows
#CorrespondingAuthorIDs=[54,109]
#CorrespondingEmails=['ben.jones@uta.edu','katherine.woodruff@uta.edu']
CorrespondingAuthorIDs=[]
CorrespondingEmails=[]

# For institution ordering, choose: 
#    1. to order institutions by author list appearance, or 
#    2. to order them alphabetically.
InstitutionOrderingScheme=2

# Author ordering determined is by ListOrder column in excel. All 1's, followed by all the 2's, 
# followed by [...] followed by everyone else, in excel sheet order.

In [114]:
#Lift from excel sheet which columns are institutions and addresses.
# Anything labeled "Institution" is an institution column and the one to its
# right is an address column.
ar=["Institution" in i for i in dat.columns]
InstColumns=np.array(dat.columns)[ar]
AddrColumns=np.array(dat.columns)[np.concatenate([[False],ar[:-1]])]
AddrDict={InstColumns[i]:AddrColumns[i] for i in range(0,len(InstColumns))}
for i in InstColumns:
    dat.loc[:,i]=dat.loc[:,i].str.strip()
    dat.loc[:,AddrDict[i]]=dat.loc[:,AddrDict[i]].str.strip()

In [115]:
# Put the author list and list of institutions in the correct order
OrderOfAuthors=[]
UniqueInstitutions=[]
Addresses=[]

UniqAuthorPositions=sorted(dat[dat.ListOrder.notna()].ListOrder.unique())
for j in UniqAuthorPositions:
    for i in list(dat[dat.ListOrder==j].index):
        OrderOfAuthors.append(i)
        for InstCol in InstColumns:
            if(dat.loc[i].notna()[InstCol] and not dat.loc[i][InstCol] in UniqueInstitutions):
                UniqueInstitutions.append(dat.loc[i][InstCol])
                Addresses.append(dat.loc[i][AddrDict[InstCol]])
for i in dat.index:
    if i not in OrderOfAuthors:
        OrderOfAuthors.append(i)
        for InstCol in InstColumns:
            if(dat.loc[i].notna()[InstCol] and not dat.loc[i][InstCol] in UniqueInstitutions):
                UniqueInstitutions.append(dat.loc[i][InstCol])
                Addresses.append(dat.loc[i][AddrDict[InstCol]])
                
UniqueInstitutions=np.array(UniqueInstitutions)
Addresses=np.array(Addresses)

 
if(InstitutionOrderingScheme==2):
    Order=np.argsort(Addresses)
    Addresses=Addresses[Order]
    UniqueInstitutions=UniqueInstitutions[Order]

def InstitutionID(Name):
    return str(np.where(UniqueInstitutions==Name)[0][0]+1)

ordered=dat.loc[OrderOfAuthors]
UniqueNotes=ordered[ordered.Footnote.notnull()].Footnote.unique()

def NoteID(Name):
    return str(num2alpha[np.where(UniqueNotes==Name)[0][0]+1])


num2alpha = dict(zip(range(1, 27), string.ascii_lowercase))

In [116]:
# Function to make the author list for IOP journals (JCAP, JHEP, JINST...)

def MakeIOPAuthorList():

    authorlist=str()
    UsedNotes=defaultdict(int)
    for a in OrderOfAuthors:
        entry=dat.loc[a]
        instids=""
        noteids=""
        notestring=""
        for InstCol in InstColumns:
            if(entry.notna()[InstCol]):
                instids+=InstitutionID(entry[InstCol])+","
        if(len(str(entry.Footnote))>5):
            nid=NoteID(entry.Footnote)
            noteids+=","+nid
            if(UsedNotes[nid]==False):
                notestring+="\\note["+nid+"]{" + entry.Footnote + "}"
            UsedNotes[nid]=UsedNotes[nid]+1
                
        authorlist+=("\\author["+instids[:-1]+noteids+"]{"+entry.Initial.strip()+"~"+entry.LastName.strip()+notestring+",}\n")
        authorlist+=("%\n")
    authorlist+=("%\n")

    authorlist+=("%\n")
    for i in range(0,len(UniqueInstitutions)):
        authorlist+="\\affiliation[" + str(InstitutionID(UniqueInstitutions[i])) + ']{\n' + UniqueInstitutions[i].replace('\\\\','\\\\\n')+", "+Addresses[i]+"}\n"
        authorlist+=("%\n")
    for email in CorrespondingEmails:
        authorlist+="\emailAdd{"+email+"}\n"

    return authorlist

In [117]:
# Function to make the author list for IOP journals (JCAP, JHEP, JINST...)

def MakeSpringerNatureAuthorList():

    authorlist=str()
    UsedNotes=defaultdict(int)
    for a in OrderOfAuthors:
        entry=dat.loc[a]
        instids=""
        noteids=""
        notestring=""
        for InstCol in InstColumns:
            if(entry.notna()[InstCol]):
                instids+=InstitutionID(entry[InstCol])+","
        if(len(str(entry.Footnote))>5):
            nid=NoteID(entry.Footnote)
            noteids+=","+nid
            if(UsedNotes[nid]==False):
                notestring+="\\note["+nid+"]{" + entry.Footnote + "}"
            UsedNotes[nid]=UsedNotes[nid]+1
        authorlist+=("\\author["+instids[:-1]+noteids+"]{"+entry.Initial.strip()+"~"+entry.LastName.strip()+notestring+",}\n")
        authorlist+=("%\n")
    authorlist+=("%\n")

    authorlist+=("%\n")
    for i in range(0,len(UniqueInstitutions)):
        authorlist+="\\affil[" + str(InstitutionID(UniqueInstitutions[i])) + ']{\n' + UniqueInstitutions[i].replace('\\\\','\\\\\n')+", "+Addresses[i]+"}\n"
        authorlist+=("%\n")


    return authorlist

In [118]:
# Function to make the author list for Elsevier journals (NIM...)

def MakeElsevierAuthorList():
    authorlist=str()
    UsedNotes=defaultdict(int)
    for a in OrderOfAuthors:
        entry=dat.loc[a]
        instids=""
        for InstCol in InstColumns:
            if(entry.notna()[InstCol]):
                instids+=InstitutionID(entry[InstCol])+","
        NotesText=""
        if(len(str(entry.Footnote))>5):
            NotesText+="\\fnref{"+NoteID(entry.Footnote)+"}"
            nid=NoteID(entry.Footnote)
            if(UsedNotes[nid]==0):
                authorlist+=('\\fntext['+NoteID(entry.Footnote)+']{' + entry.Footnote.strip() + '}\n')
            UsedNotes[nid]=UsedNotes[nid]+1
        if(a in CorrespondingAuthorIDs):
            NotesText+="\\corref{cor}"
        authorlist+=("\\author["+instids[:-1]+"]{"+entry.Initial.strip()+"~"+entry.LastName.strip()+NotesText+"}\n")
        authorlist+=("%\n")
    authorlist+=("%\n")
    for i in range(0,len(UniqueInstitutions)):
        authorlist+="\\address[" + InstitutionID(UniqueInstitutions[i]) + ']{\n' + UniqueInstitutions[i].replace('\\\\','\\\\\n')+", "+ Addresses[i].strip()+"}\n"
        authorlist+=("%\n")
    if(len(CorrespondingAuthorIDs)>0):
        authorlist+="\\cortext[cor]{Corresponding Authors}"
    return authorlist

In [119]:
# Function to make the author list for APS journals (PRD, PRL...)

def MakeAPSAuthorList():
    authorlist=str()
    for a in OrderOfAuthors:
        entry=dat.loc[a]
        authorlist+="\\author{"+entry.Initial.strip()+"~"+entry.LastName.strip()+"}\n"
        for InstCol in InstColumns:
            if(entry.notna()[InstCol]):
                authorlist+="\\affiliation{"+entry[InstCol].replace('\\\\','\\\\\n')+", "+entry[AddrDict[InstCol]]+"}\n"
        if(len(str(entry.Footnote))>5):
            authorlist+=('\\thanks{' + entry.Footnote.strip() + '}\n')
        if(a in CorrespondingAuthorIDs):
            authorlist+=('\\thanks{Corresponding Author}\n')


        authorlist+=("%\n")
    return authorlist

In [120]:
# Function to make the author list for arXiv

def MakeArXivAuthorList():
    authorlist=str("NEXT Collaboration: ")
    for a in OrderOfAuthors[:-1]:
        entry=dat.loc[a]
        authorlist+=entry.Initial.strip()+" "+entry.LastName.strip()+", "
    entry=dat.loc[OrderOfAuthors[-1]]
    authorlist=authorlist[:-2]+" and " + entry.Initial.strip()+" "+entry.LastName.strip()
    return authorlist

In [121]:
#Function to make the author list in plain text

def MakeHTMLAuthorList():
    authorlist=""
    for i in range(0,len(UniqueInstitutions)):
        #authorlist+="#"+latex2text.latex2text(Addresses[i])+"\n\n "
        ThisInst=dat[dat.Institution1==UniqueInstitutions[i]]
        if(len(ThisInst)>0):
            authorlist+="<b>"+(LatexNodes2Text().latex_to_text(UniqueInstitutions[i]))+"</b><br>\n"
            for i in range(0,len(ThisInst)):
                authorlist+=LatexNodes2Text().latex_to_text(ThisInst.iloc[i].Initial.strip())+" "+LatexNodes2Text().latex_to_text(ThisInst.iloc[i].LastName.strip())+", "
            authorlist=authorlist[:-2]+"<br><br>\n  "
            authorlist+="\n"
    return authorlist

In [122]:
# Make author lists and store to .tex

f=open("authors_elsevier.tex",'w')
f.write(MakeElsevierAuthorList())
f.close()

f=open("authors_iop.tex",'w')
f.write(MakeIOPAuthorList())
f.close()

f=open("authors_aps.tex",'w')
f.write(MakeAPSAuthorList())
f.close()

f=open("authors_springernature.tex",'w')
f.write(MakeSpringerNatureAuthorList())
f.close()

f=open("authors_arxiv.tex",'w')
f.write(MakeArXivAuthorList())
f.close()


f=open("authors_html.html",'w')
f.write(MakeHTMLAuthorList())
f.close()


In [123]:
authorlist=str()
UsedNotes=defaultdict(int)
for a in OrderOfAuthors:
    entry=dat.loc[a]
    instids=""
    for InstCol in InstColumns:
        if(entry.notna()[InstCol]):
            instids+=InstitutionID(entry[InstCol])+","
    NotesText=""
    if(len(str(entry.Footnote))>5):
        NotesText+="\\fnref{"+NoteID(entry.Footnote)+"}"
        nid=NoteID(entry.Footnote)
        if(UsedNotes[nid]==0):
            authorlist+=('\\fntext['+NoteID(entry.Footnote)+']{' + entry.Footnote.strip() + '}\n')
        UsedNotes[nid]=UsedNotes[nid]+1
    if(a in CorrespondingAuthorIDs):
        NotesText+="\\corref{cor}"
    authorlist+=("\\author["+instids[:-1]+"]{"+entry.Initial.strip()+"~"+entry.LastName.strip()+NotesText+"}\n")
    authorlist+=("%\n")
authorlist+=("%\n")
for i in range(0,len(UniqueInstitutions)):
    authorlist+="\\address[" + InstitutionID(UniqueInstitutions[i]) + ']{\n' + UniqueInstitutions[i].replace('\\\\','\\\\\n')+", "+ Addresses[i].strip()+"}\n"
    authorlist+=("%\n")
if(len(CorrespondingAuthorIDs)>0):
    authorlist+="\\cortext[cor]{Corresponding Authors}"